In [ ]:
STEP_1   ----> importing all the libraries

In [1]:
import os 
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten 
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

In [3]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
STEP-2---->Loading the dataset (images in this case) with the help of a helper function that  will help resize it,as well as load images in workspace

In [5]:
image_size=250
labels = ['PNEUMONIA', 'NORMAL']

In [6]:
#Helper fn
def data_loader(data_dir):
    data = list()
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (image_size, image_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
                
    return np.array(data)

In [7]:
train = data_loader('../input/chest-xray-pneumonia/chest_xray/chest_xray/train')
test = data_loader('../input/chest-xray-pneumonia/chest_xray/chest_xray/test')
val = data_loader('../input/chest-xray-pneumonia/chest_xray/chest_xray/val')

In [9]:
print(len(train))
print(len(test))
print(len(val))

SHOWING AN IMAGE USING OpenCV lib ------to visualise 

In [11]:
##showing one ramdom image 
import cv2 as cv

In [12]:
img = cv.imread("../input/chest-xray-pneumonia/chest_xray/train/NORMAL/IM-0115-0001.jpeg")
img.shape

In [13]:
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)


STEP-3--->SPLITTING THE DATASET INTO TRAIN AND TEST DATA

In [14]:
## dataset splitting
X_train,y_train = [],[]
X_test,y_test   = [],[]
X_val,y_val=[],[]
for feature, label in train:
    X_train.append(feature)
    y_train.append(label)
for feature, label in test:
    X_test.append(feature)
    y_test.append(label)
for feature, label in val:
    X_val.append(feature)
    y_val.append(label)

In [15]:
print(np.asarray(X_train).shape)
print(np.asarray(X_test).shape)
print(np.asarray(y_train).shape)
print(np.asarray(y_test).shape)
print(np.asarray(X_val).shape)
print(np.asarray(y_val).shape)


STEP-4---->NORMALIZATION OF IMAGES
This step is done to normalize the pixel values of images between 0 and 1.By doing so,we can cut the values that are potential outliers(i.e either too high or too low).
To normalize an image,,,,,,img=img/255

In [16]:
## data normalization
X_train = np.array(X_train) / 255
y_train = np.array(y_train)
X_test = np.array(X_test) / 255
y_test = np.array(y_test)
X_val=np.array(X_val)/255
y_val=np.array(y_val)

In [17]:
type(X_train)

While reshaping,The numbers mean [batch_size, height, width, channels].                                                             
-1 means that the length in the dimension is inferred so we don’t have to specify it.                                               
1 means we’re using a black and white picture so we’ll only have one layer image(i.e. it is grayscale image).

In [18]:
X_train = X_train.reshape(-1, image_size, image_size, 1)
y_train = np.array(y_train)

X_val = X_val.reshape(-1, image_size, image_size, 1)
y_val = np.array(y_val)

X_test = X_test.reshape(-1, image_size, image_size, 1)
y_test = np.array(y_test)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

After converting images to numpy array which is continous data ,we need to convert it to categorical data(0 and 1 specifically),therefore we'll use one hot encoding ..                                                                               
By using numeric values, we more easily determine a probability for our values.

In [21]:
from tensorflow.keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [22]:
## Model 1 defintion

model1 = Sequential()
model1.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu' , input_shape = (250,250,1)))
model1.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D((2,2) , strides = 2 , padding = 'valid'))

model1.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.1))
model1.add(MaxPool2D((2,2) , strides = 2 , padding = 'valid'))

model1.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(MaxPool2D((2,2) , strides = 2 , padding = 'valid'))

model1.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
#model1.add(BatchNormalization())
model1.add(Conv2D(128 , (3,3) , strides = 1 , padding ='valid' , activation = 'relu'))
model1.add(BatchNormalization())
#model1.add(Dropout(0.1))
model1.add(MaxPool2D((2,2) , strides = 2 , padding = 'valid'))

model1.add(Conv2D(256 , (3,3) , strides = 1 , padding ='valid' , activation = 'relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D((2,2) , strides = 2 , padding ='valid'))

model1.add(Conv2D(512 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
#model1.add(BatchNormalization())
#model1.add(Conv2D(512 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'))
model1.add(BatchNormalization())
#model1.add(Dropout(0.1))
model1.add(MaxPool2D((2,2) , strides = 2 , padding = 'valid'))

model1.add(Flatten())
model1.add(Dense(units = 256 , activation = 'relu'))
model1.add(Dense(units = 64, activation ='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(units = 16, activation ='relu'))
model1.add(Dense(units = 1 , activation = 'sigmoid'))




In [23]:
model1.compile(optimizer = "rmsprop" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model1.summary()
print('\n')

The plot_model() function in Keras will create a plot of your network. This function takes a few useful arguments:

In [24]:
## plotting model architechture
from keras.utils.vis_utils import plot_model
plot_model(model1, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)

In [33]:
y_predict_cnn = model1.predict(X_test)
y_predict_cnn


In [34]:
def convert_to_label(y_predict):
    ans = []
    for pair in y_predict:
        if pair[0]>0.5:
            ans.append(0)
        else:
            ans.append(1)
    return np.asnumpy(ans)


In [35]:
y_predict_cnn = convert_to_label(y_predict_cnn)
print(len(y_predict_cnn))
print(type(y_predict_cnn))
print(y_predict_cnn)
